In [1]:
server = 'irc.chat.twitch.tv'
port = 6667
nickname = 'mxnn21'
token = 'oauth:lj5n5gzwyamz3gu4hx7p3157mes8sm'
channel = '#kyedae_'

In [2]:
import socket
sock = socket.socket()

In [3]:
sock.connect((server,port))

In [4]:
sock.send(f"PASS {token}\n".encode('utf-8'))
sock.send(f"NICK {nickname}\n".encode('utf-8'))
sock.send(f"JOIN {channel}\n".encode('utf-8'))

14

In [5]:
resp = sock.recv(2048).decode('utf-8')
resp

':tmi.twitch.tv 001 mxnn21 :Welcome, GLHF!\r\n:tmi.twitch.tv 002 mxnn21 :Your host is tmi.twitch.tv\r\n:tmi.twitch.tv 003 mxnn21 :This server is rather new\r\n:tmi.twitch.tv 004 mxnn21 :-\r\n:tmi.twitch.tv 375 mxnn21 :-\r\n:tmi.twitch.tv 372 mxnn21 :You are in a maze of twisty passages, all alike.\r\n:tmi.twitch.tv 376 mxnn21 :>\r\n:mxnn21!mxnn21@mxnn21.tmi.twitch.tv JOIN #kyedae_\r\n:mxnn21.tmi.twitch.tv 353 mxnn21 = #kyedae_ :mxnn21\r\n:mxnn21.tmi.twitch.tv 366 mxnn21 #kyedae_ :End of /NAMES list\r\n'

In [ ]:
#sock.close()

In [6]:
import logging
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s — %(message)s',
                    datefmt='%Y-%m-%d_%H:%M:%S',
                    handlers=[logging.FileHandler('chat.log', encoding='utf-8')])

In [7]:
logging.info(resp)

In [10]:
from emoji import demojize
while True:
    resp = sock.recv(2048).decode('utf-8')
    if resp.startswith('PING'):
        sock.send("PONG\n".encode('utf-8'))
    elif len(resp)>0:
        logging.info(demojize(resp))

KeyboardInterrupt: 

In [14]:
import pandas as pd

def get_chat_dataframe(file):
    data = []

    with open(file, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n\n\n')
        
        for line in lines:
            try:
                time_logged = line.split('—')[0].strip()
                time_logged = datetime.strptime(time_logged, '%Y-%m-%d_%H:%M:%S')

                username_message = line.split('—')[1:]
                username_message = '—'.join(username_message).strip()

                username, channel, message = re.search(
                    ':(.*)\!.*@.*\.tmi\.twitch\.tv PRIVMSG #(.*) :(.*)', username_message
                ).groups()

                d = {
                    'dt': time_logged,
                    'channel': channel,
                    'username': username,
                    'message': message
                }

                data.append(d)
            
            except Exception:
                pass
            
    return pd.DataFrame().from_records(data)
        
    
df = get_chat_dataframe('chat.log')

In [15]:
df.set_index('dt', inplace=True)

print(df.shape)

df.head()

(5, 3)


,channel,username,message
dt,,,
2020-12-29 20:21:23,kyedae_,mxnn21,API testing...API testing hola
2020-12-29 20:22:05,kyedae_,mxnn21,Damn! chat log script works!
2020-12-29 20:22:27,kyedae_,mxnn21,testing twitch global emotes: PogChamp
2020-12-29 20:22:49,kyedae_,mxnn21,testing sub emotes: kyedaeRAGE kyedaeDX
2020-12-29 20:22:58,kyedae_,mxnn21,damn it all works woohooo
